In [135]:
import pandas as pd
import numpy as np
from torch import nn
from sklearn.preprocessing import PowerTransformer
from torchviz import make_dot
from itertools import islice
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import statistics as stats
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import hiddenlayer as hl

# Importing the data

In [136]:
def to_list(str):
    '''
    :param str: string representing a list of centipawn losses
    :return: list of integer centipawn losses
    '''
    string = str.replace('[','').replace(']','')
    ls = string.split(',')
    list = [int(i) for i in ls]

    return list

In [137]:
def process(eval):
    '''
    :param eval: list of integer centipawn losses
    :return: array of lists of [evaluation, centipawn loss]
    '''

    # starting evaluation of 30 centipawns
    sum = 30

    i = 0
    res = []

    # iterating through centipawn losses
    for cpl in eval:

        # subtracting the cpl for white's moves
        if i % 2 ==0:
            res.append([sum,cpl])
            sum -= cpl
            i += 1

        # adding the cpl for black's moves
        else:
            sum+=cpl
            i+=1

    return np.array(res)

In [138]:
# reading *some* of the data
dfs = []

players = ['andreikin, dmitry', 'anand, viswanathan', 'wang, hao', 'grischuk, alexander', 'karjakin, sergey','duda, jan-krzysztof', 'radjabov, teimour', 'dominguez perez, leinier','nakamura, hikaru', 'vachier-lagrave, maxime','aronian, levon','mamedyarov, shakhriyar', 'so, wesley','ding, liren', 'rapport, richard', 'nepomniachtchi, ian', 'giri, anish', 'firouzja, alireza', 'caruana, fabiano','carlsen, magnus','zelcic, robert','khotenashvili, bela', 'bischoff, klaus', 'hoffmann, asa','kaufman, lawrence','bellaiche, elise']

# reading the csvs
for player in players:
    df = pd.read_csv('blitz/'+player +'.csv')
    dfs.append(df)
df = pd.concat(dfs)


print(f"Total  games: {len(df)}")

# Filtering out * values
df = df[df['WhiteELO'] != '*']
df = df[df['BlackELO'] != '*']
df[['WhiteELO', 'BlackELO']] = df[['WhiteELO', 'BlackELO']] .astype(int)

df = df[df['Eval'] != '']
df = df[ df['Eval'].apply(lambda x: isinstance(x, str))]


# converting the evaluation to a list
df['Eval'] = df['Eval'].apply( to_list)
df['Eval'] = df['Eval'].apply( process )

print(f"Evaluted games: {len(df['Eval'])}")

Total  games: 4210
Evaluted games: 434


In [139]:
x = np.array(df['Eval'])
length = np.array(df['Eval'].apply(len))

In [140]:
# creating and fitting a power transformer
pt = PowerTransformer()
y = np.concatenate(x)
pt.fit(y)
transformed = pt.transform(y)

In [141]:
# need a function now to (effieciently) change these to lists of length list
transformed_array = [np.array(list(islice(iter(transformed), elem)))
        for elem in length]

In [142]:
print(f"Unique evaluated games: {df['Game'].nunique()}")

Unique evaluated games: 422


# Preparing the data for the Neural Net

In [143]:
# converting evaluations and length to tensors
evals = [torch.tensor(i, dtype = torch.float32) for i in transformed_array]
lengths = [len(tensor) for tensor in evals]

In [144]:
# Padding my sequences - not sure why batch first works, but it does
#inputs = torch.nn.utils.rnn.pad_sequence(evals, batch_first=True, padding_value=0.0)

In [145]:
#inputs_array = np.array(inputs.tolist())

In [146]:
#print(inputs.shape)

In [147]:
#inputs_list =inputs.tolist()

# normalizing... a bit hacky
#inputs_array = (np.array(inputs_list) - np.array(inputs_list).mean())/ np.linalg.norm(np.array(inputs_list))

In [148]:
def normalize(array):
    '''
    :param array:
    :return:
    '''
    return (array - array.mean())/array.std()

def denormalize(array, value):
    '''
    :param array:
    :param value:
    :return:
    '''
    return value*array.std() + array.mean()

In [149]:
#print(df['WhiteELO'].value_counts())

In [150]:
# Converting White and Black's ELOs to tensors
white_elo_arr = np.array(df['WhiteELO'])

white_elo = normalize(white_elo_arr)

#print(white_elo)
white_elo = [torch.tensor(i, dtype = torch.float32) for i in white_elo]



black_elo = np.array(df['BlackELO'])
black_elo = [torch.tensor(i, dtype = torch.float32) for i in black_elo]


# splitting into train and test
lengths_train, lengths_test,eval_train, eval_test, black_train, black_test, white_train, white_test  = train_test_split(lengths, evals, black_elo, white_elo, test_size=0.2,random_state=0, shuffle = True)


In [151]:
# zipping the elo together with the evaluations
train_data_zip = list(zip(eval_train, white_train))
test_data_zip = list(zip(eval_test, white_test))

In [152]:
black_elo = torch.stack(black_elo)
white_elo = torch.stack(white_elo)

# Creating the Neural Net

In [153]:
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, no_layers):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.no_layers = no_layers
        self.rnn = nn.RNN(input_size, hidden_size, no_layers, batch_first = True, bias = True)
        self.fc = nn.Linear(hidden_size,1, bias = False)
        self.final = nn.Tanh()

    def forward(self, x):
        t, l = torch.nn.utils.rnn.pad_packed_sequence(x)

        # initial hidden state
        #h0 = torch.rand(self.no_layers,batch_size,self.hidden_size)

        out, _ = self.rnn(x)


        # shape batches, seq_length, hidden_size
        output ,lengths = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first = True)

        out = [output[e, i-1,:].unsqueeze(0) for e, i in enumerate(lengths)]

        out = torch.cat(out, dim = 0)
        #print(out.shape)
        #print("Linear weights", self.fc.weight)


        out = self.fc(out)
        out = self.final(out)
        #print(out.shape)

        return out[:,0]


    #def init_hidden(self):
    #    return nn.init.kaiming_uniform_(torch.empty(1, self.hidden_size))

# need to figure out exactly how the dimensions changed


In [154]:
class MyCollator(object):
    '''
    Yields a batch from a list of Items
    Args:
    test : Set True when using with test data loader. Defaults to False
    percentile : Trim sequences by this percentile
    '''

    # remove that eventually. I'm going to need to make my dataset a tuple with evals and elo
    #def __init__(self):

    def __call__(self, batch):
        data = [item[0] for item in batch]
        target = [item[1] for item in batch]
        lens = [i.shape[0] for i in data]

        #print(lens)
        data = torch.nn.utils.rnn.pad_sequence(data, batch_first=True,padding_value = 0)
        #data = data.unsqueeze(2)
        #print("PADDED",data)
        evals_packed = torch.nn.utils.rnn.pack_padded_sequence(data,batch_first = True, lengths=lens,enforce_sorted=False)

        target = torch.tensor(target,dtype=torch.float32)
        return [evals_packed,target]

In [155]:
input_size = 2
hidden_size = 20
no_layers = 3
batch_size = 5

In [156]:
# (defining my model)
model = MyRNN(input_size, hidden_size, no_layers)
collate = MyCollator()

In [157]:
# for name, param in model.named_parameters():
#     print('name: ', name)
#     print(type(param))
#     print('param.shape: ', param.shape)
#     print('param.requires_grad: ', param.requires_grad)
#     print('=====')

In [158]:
#eval_train = [torch.tensor(i, dtype = torch.float32) for i in eval_train]
#white_train = [torch.tensor(i, dtype = torch.float32) for i in white_train]

#eval_train = torch.stack(eval_train)
#white_train = torch.stack(white_train)

#train_data = list(zip(evals_packed, white_train))


#eval_test = [torch.tensor(i, dtype=torch.float32) for i in eval_test]
#white_test = [torch.tensor(i, dtype=torch.float32) for i in white_test]

#eval_test = torch.stack(eval_test)
#white_test = torch.stack(white_test)


# not sure why I'm seeing a warning...

In [159]:
print(model.parameters)

<bound method Module.parameters of MyRNN(
  (rnn): RNN(2, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=1, bias=False)
  (final): Tanh()
)>


# Training the model

In [160]:
data_loader = torch.utils.data.DataLoader(train_data_zip, batch_size=batch_size, shuffle=True ,collate_fn=collate)

In [161]:
learning_rate = .2

In [162]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [163]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [164]:
# OK I've figured out the issue, I also need the sequence length for the RNN

In [ ]:
avg_losses = []
epochs = []
avg_loss = 1

for epoch in range(100):

    if (epoch+1) % 10 ==0:
        learning_rate /= 2
        optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

    i = 0
    losses = []
    for evals, elo in data_loader:
        evals = evals.to(device)

        elo = elo.to(device)
        i +=1
        outputs = model(evals)
        #print(outputs)
        #print(outputs.shape, elo.shape)
        loss = criterion(outputs,elo)

        # optimizing
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    change = stats.mean(losses)/avg_loss
    avg_loss = stats.mean(losses)
    avg_losses.append(avg_loss)
    epochs.append(epoch)
    print(f'Epoch {epoch+1} step {i+1} - Learning Rate : {learning_rate}- Avg Loss: {avg_loss:3f} - Change in loss: {change}')

Epoch 1 step 71 - Learning Rate : 0.2- Avg Loss: 0.987626 - Change in loss: 0.9876262684485742
Epoch 2 step 71 - Learning Rate : 0.2- Avg Loss: 0.968867 - Change in loss: 0.9810052528514919
Epoch 3 step 71 - Learning Rate : 0.2- Avg Loss: 0.978322 - Change in loss: 1.0097590331161888
Epoch 4 step 71 - Learning Rate : 0.2- Avg Loss: 0.966762 - Change in loss: 0.9881843884216049
Epoch 5 step 71 - Learning Rate : 0.2- Avg Loss: 0.966832 - Change in loss: 1.0000721153307741


# Why are 4/5 always zero???

In [ ]:
plt.scatter(epochs, avg_losses)

## Test data

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

In [ ]:
test_data_loader = torch.utils.data.DataLoader(test_data_zip, batch_size=1, shuffle=False ,collate_fn=collate)

In [ ]:
losses = []
outputs = []
elos = []
for evals, elo in test_data_loader:
    #print("evals",evals.shape)
    evals = evals.to(device)
    elo = elo.to(device)
    output = model(evals)
    outputs.append(output.item())
    elos.append(elo.item())
    loss = criterion(output,elo)
    #print(f'Model prediction : {output} \n ELO : {elo} \n MSE : {loss}')
    losses.append(loss.item())
print(f'Average loss : {stats.mean(losses)}')

In [ ]:
model(evals)

In [ ]:
plt.scatter(elos,outputs, alpha = 1)

# Plotting the neural net architecture

In [ ]:
# note - to use any of these, I'm not sure that I can use the packed sequence. I would have to change the way the NN works, a little bit
tb = SummaryWriter()

In [ ]:
print(max(outputs))
print(min(outputs))

In [ ]:
# note - doesn't support packed stuff
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [ ]:
images, labels = next(iter(test_data_loader))

In [ ]:
print(images)

In [ ]:
tb.add_graph(model)

In [ ]:
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, no_layers):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.no_layers = no_layers
        self.rnn = nn.RNN(input_size, hidden_size, no_layers, batch_first = True, bias = False)
        self.fc = nn.Linear(hidden_size,1, bias = False)

    def forward(self, x):
        # initial hidden state
        h0 = torch.rand(self.no_layers,x.size(0),self.hidden_size)

        out, _ = self.rnn(x,h0)
        #print(out) #return out
        out = out[:,-1,:]
        #print(out)
        out = self.fc(out)
        #print(out)
        m = nn.Sigmoid()
        out = m(out)
        return out
        # print("out: ",out)
        # print('RNN WEIGHTS: ',self.rnn.weight_ih_l0  )
        # print(self.fc.weight)
        #
        # #print("Count these...",out)
        # m = nn.Sigmoid()
        # out = m(out)
        # if x.size(0) == 1:
        #     return out[0]
        # else:
        #     return torch.squeeze(out)


In [ ]:
model = MyRNN(input_size, hidden_size, no_layers)